<a href="https://colab.research.google.com/github/ShaliniAnandaPhD/PIXEL-PIONEERS-TUTORIALS/blob/main/Stability_AI_Interactive_Art_generate_layered_prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Notebook Instructions and Benefits

1. **Initial Setup**: Install the Stability SDK using `%pip install stability-sdk` and set up your environment with your API key. This prepares your notebook for interacting with the Stability AI.

2. **Generate Art with Custom Prompts**: Use the `generate_art` function to create images based on your textual prompts. This function combines your creative input with the AI's capabilities, allowing for the generation of unique art.

3. **Detailed Prompt Crafting**: Utilize the interactive widgets for crafting detailed prompts. You can specify the scene, colors, emotions, style, and key elements. This approach helps in formulating effective prompts that lead to more accurate and aesthetically pleasing AI-generated images.

4. **View and Save Generated Images**: Interact with the UI elements to generate art and view the results instantly. You can save your favorite images and view them in a gallery format. This feature makes it easy to keep track of and revisit your generated artworks.

5. **Iterative Prompt Refinement**: The notebook encourages an iterative process. Based on the AI-generated results, you can refine your prompts for better outcomes, learning about the AI’s strengths and limitations in the process.

### How It Benefits the User:

- **Creative Exploration**: Users can explore their creativity by crafting various prompts and seeing them visualized as artworks.
- **Understanding AI Capabilities**: Users gain insights into how AI interprets textual prompts and learn to craft more effective prompts over time.
- **Educational Value**: The notebook serves as an educational tool for understanding the interaction between human input and AI output in the realm of art generation.
- **Personalized Art Creation**: Users can create personalized artworks that can be used for various purposes, including digital content, inspiration, or simply as a fun creative exercise.
- **Experimentation and Learning**: The iterative nature of the process encourages experimentation, allowing users to learn and improve their prompt crafting skills.

This notebook offers a hands-on experience with AI-generated art, combining creativity with technology to produce unique visual representations of textual prompts.

From Stability AI documentation

In [1]:
%pip install stability-sdk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.60.0
    Uninstalling grpcio-1.60.0:
      Successfully uninstalled grpcio-1.60.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 which is incompatible.


FROM STABILITY AI documentation

In [6]:
import getpass, os
import io
import warnings
from IPython.display import display
from PIL import Image
from stability_sdk import client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation

# Set up environment variables
os.environ['STABILITY_HOST'] = 'grpc.stability.ai:443'
os.environ['STABILITY_KEY'] = getpass.getpass('api key')

# Initialize Stability API client
stability_api = client.StabilityInference(
    key=os.environ['STABILITY_KEY'],
    verbose=True,
    engine="stable-diffusion-xl-1024-v1-0"
)


api key··········


INFO:stability_sdk.client:Opening channel to grpc.stability.ai:443
INFO:stability_sdk.client:Channel opened to grpc.stability.ai:443


In [17]:
from PIL import Image
import io

def generate_art(prompt, width, height, steps, cfg_scale):
    answers = stability_api.generate(
        prompt=prompt,
        steps=steps,
        cfg_scale=cfg_scale,
        width=width,
        height=height,
        samples=1,
        sampler=generation.SAMPLER_K_DPMPP_2M
    )

    for resp in answers:
        for artifact in resp.artifacts:
            if artifact.finish_reason == generation.FILTER:
                warnings.warn("Content filter activated. Adjust the prompt.")
            elif artifact.type == generation.ARTIFACT_IMAGE:
                # Return a PIL Image object
                return Image.open(io.BytesIO(artifact.binary))
    # Return None explicitly if no image is generated
    return None

In [20]:
import ipywidgets as widgets
from IPython.display import clear_output, display
from PIL import Image
import io
import datetime
import os

# UI elements for customizable parameters
width_slider = widgets.IntSlider(value=1024, min=256, max=1024, step=64, description='Width:')
height_slider = widgets.IntSlider(value=1024, min=256, max=1024, step=64, description='Height:')
steps_slider = widgets.IntSlider(value=50, min=20, max=100, step=5, description='Steps:')
cfg_scale_slider = widgets.FloatSlider(value=8.0, min=1.0, max=11.0, step=0.1, description='CFG Scale:')

# Dropdown for predefined themes
theme_dropdown = widgets.Dropdown(
    options=['Custom', 'Futuristic City', 'Underwater World', 'Fantasy Landscape', 'Space Colony'],
    value='Custom',
    description='Theme:',
)

# Textarea for custom prompts
prompt_input = widgets.Textarea(
    placeholder='Type your prompt here or select a theme',
    description='Prompt:',
    layout=widgets.Layout(width='100%', height='100px')
)

# Button to generate art
generate_button = widgets.Button(
    description='Generate Art',
    button_style='info',
    icon='check'
)

# Output area to display the generated art
output_area = widgets.Output()

# Gallery to display thumbnails of generated images
gallery_area = widgets.VBox([])

# Function to handle theme selection and prompt input
def handle_theme_change(change):
    if change.new != 'Custom':
        prompt_input.value = change.new

theme_dropdown.observe(handle_theme_change, names='value')

# Function to save the generated image
def save_image(img):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"generated_image_{timestamp}.png"
    img.save(filename)
    return filename

# Function to display a full-size image when a thumbnail is clicked
def on_thumbnail_clicked(b):
    with output_area:
        clear_output(wait=True)
        img = open(b.filename, "rb").read()
        display(widgets.Image(value=img, format='png', width=500, height=500))

# Function to add a thumbnail to the gallery
def add_to_gallery(filename):
    thumbnail = widgets.Image(value=open(filename, "rb").read(), format='png', width=100, height=100)
    button = widgets.Button(description="View", button_style='primary')
    button.filename = filename
    button.on_click(on_thumbnail_clicked)
    gallery_item = widgets.HBox([thumbnail, button])
    gallery_area.children = list(gallery_area.children) + [gallery_item]

# Function to display and save generated image
def display_and_save_image(img):
    with output_area:
        display(img)
    filename = save_image(img)
    add_to_gallery(filename)

# Function to handle button click
def on_generate_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        prompt = prompt_input.value if theme_dropdown.value == 'Custom' else theme_dropdown.value
        img = generate_art(prompt, width_slider.value, height_slider.value, steps_slider.value, cfg_scale_slider.value)
        if img is not None:
            display_and_save_image(img)
        else:
            print("No image was generated. Please try a different prompt.")

generate_button.on_click(on_generate_button_clicked)

# Layout for UI elements
ui_layout = widgets.Layout(
    display='flex',
    flex_flow='row wrap',
    align_items='flex-start',
    width='100%'
)

# Display the UI
ui_elements = widgets.VBox([
    theme_dropdown,
    prompt_input,
    width_slider,
    height_slider,
    steps_slider,
    cfg_scale_slider,
    generate_button,
    output_area
], layout=ui_layout)

gallery_layout = widgets.Layout(
    display='flex',
    flex_flow='row wrap',
    align_items='flex-start',
    width='100%'
)

# Combine UI elements and gallery
combined_ui = widgets.VBox([ui_elements, widgets.HTML('<h3>Generated Image Gallery:</h3>'), gallery_area], layout=gallery_layout)

display(combined_ui)



Advanced prompting for Art generation

In [30]:
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import clear_output

def generate_art(prompt, width, height, steps, cfg_scale):
    # Existing Stability AI code
    answers = stability_api.generate(
        prompt=prompt,
        steps=steps,
        cfg_scale=cfg_scale,
        width=width,
        height=height,
        samples=1,
        sampler=generation.SAMPLER_K_DPMPP_2M
    )

    for resp in answers:
        for artifact in resp.artifacts:
            if artifact.finish_reason == generation.FILTER:
                warnings.warn("Content filter activated. Adjust the prompt.")
            elif artifact.type == generation.ARTIFACT_IMAGE:
                return Image.open(io.BytesIO(artifact.binary))
    return None

# HTML formatted instructions for crafting effective prompts
instruction_html = """
<h4>Guidelines for Crafting Detailed Prompts:</h4>
<ul>
    <li><b>Scene Description:</b> Describe the main setting or scene. Be specific about location, time of day, and atmosphere.</li>
    <li><b>Colors and Emotions:</b> Specify the color palette and the emotional tone of the image. Think about how you want the viewer to feel.</li>
    <li><b>Style or Artist:</b> Mention if you want the image in a particular artistic style or inspired by a specific artist.</li>
    <li><b>Key Elements:</b> List any specific objects, characters, or elements that should be included in the image.</li>
</ul>
<p><i>Example Prompt:</i> 'A serene mountain landscape at dawn, with soft pastel colors, in the style of Claude Monet, featuring a small tranquil lake.'</p>
"""

instructions_html_widget = widgets.HTML(value=instruction_html)

# Interactive elements for detailed prompt crafting
scene_description = widgets.Textarea(description='Scene:', placeholder='e.g., "A serene mountain landscape at dawn"')
colors_emotions = widgets.Text(description='Colors/Emotions:', placeholder='e.g., "Soft pastel colors, tranquil"')
style_artist = widgets.Text(description='Style/Artist:', placeholder='e.g., "In the style of Claude Monet"')
key_elements = widgets.Text(description='Key Elements:', placeholder='e.g., "Featuring a small tranquil lake"')

# Button to combine inputs and generate the prompt
combine_button = widgets.Button(description='Generate Prompt', button_style='success')

# Output area for the combined prompt
combined_prompt_output = widgets.Output()

# Function to combine inputs into a prompt
def on_combine_button_clicked(b):
    combined_prompt = f"{scene_description.value}, {colors_emotions.value}, {style_artist.value}, {key_elements.value}"
    with combined_prompt_output:
        clear_output(wait=True)
        print("Generated Prompt:", combined_prompt)
        prompt_input.value = combined_prompt  # Update the main prompt input

combine_button.on_click(on_combine_button_clicked)

# Display the interactive elements for prompt crafting
prompt_crafting_ui = widgets.VBox([
    instructions_html_widget,
    scene_description,
    colors_emotions,
    style_artist,
    key_elements,
    combine_button,
    combined_prompt_output
])

display(prompt_crafting_ui)



Enhanced UI with detailed prompt crafting

In [31]:
import ipywidgets as widgets
from IPython.display import clear_output, display
import datetime
import os

# Textarea for custom prompts (updated by Cell 3)
prompt_input = widgets.Textarea(
    placeholder='Your crafted prompt will appear here',
    description='Prompt:',
    layout=widgets.Layout(width='100%', height='100px'),
    disabled=True  # Disabled as it's updated by Cell 3
)

# Button to generate art
generate_button = widgets.Button(
    description='Generate Art',
    button_style='info',
    icon='check'
)

# Output area to display the generated art
output_area = widgets.Output()

# Gallery to display thumbnails of generated images
gallery_area = widgets.VBox([])

# Function to save the generated image
def save_image(img):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"generated_image_{timestamp}.png"
    img.save(filename)
    return filename

# Function to display a full-size image when a thumbnail is clicked
def on_thumbnail_clicked(b):
    with output_area:
        clear_output(wait=True)
        img = open(b.filename, "rb").read()
        display(widgets.Image(value=img, format='png', width=500, height=500))

# Function to add a thumbnail to the gallery
def add_to_gallery(filename):
    thumbnail = widgets.Image(value=open(filename, "rb").read(), format='png', width=100, height=100)
    button = widgets.Button(description="View", button_style='primary')
    button.filename = filename
    button.on_click(on_thumbnail_clicked)
    gallery_item = widgets.HBox([thumbnail, button])
    gallery_area.children = list(gallery_area.children) + [gallery_item]

# Function to display and save generated image
def display_and_save_image(img):
    with output_area:
        display(img)
    filename = save_image(img)
    add_to_gallery(filename)

# Function to handle button click for generating art
def on_generate_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        img = generate_art(prompt_input.value, width_slider.value, height_slider.value, steps_slider.value, cfg_scale_slider.value)
        if img is not None:
            display_and_save_image(img)
        else:
            print("No image was generated. Please try a different prompt.")

generate_button.on_click(on_generate_button_clicked)

# Layout for UI elements
ui_layout = widgets.Layout(
    display='flex',
    flex_flow='row wrap',
    align_items='flex-start',
    width='100%'
)

# Display the UI
ui_elements = widgets.VBox([
    theme_dropdown,
    prompt_input,
    widgets.Label("Your detailed prompt from above will be used for image generation."),
    width_slider,
    height_slider,
    steps_slider,
    cfg_scale_slider,
    generate_button,
    output_area,
    widgets.HTML('<h3>Generated Image Gallery:</h3>'),
    gallery_area
], layout=ui_layout)

display(ui_elements)



Rewritng the prompt in a way Stability AI can understand

In [34]:
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import clear_output, display, HTML

def generate_art(prompt, width, height, steps, cfg_scale):
    # Existing Stability AI code
    answers = stability_api.generate(
        prompt=prompt,
        steps=steps,
        cfg_scale=cfg_scale,
        width=width,
        height=height,
        samples=1,
        sampler=generation.SAMPLER_K_DPMPP_2M
    )

    for resp in answers:
        for artifact in resp.artifacts:
            if artifact.finish_reason == generation.FILTER:
                warnings.warn("Content filter activated. Adjust the prompt.")
            elif artifact.type == generation.ARTIFACT_IMAGE:
                return Image.open(io.BytesIO(artifact.binary))
    return None

# Instructions for crafting effective prompts
instruction_html = """
<h4>Tips for Crafting Effective Prompts:</h4>
<ul>
    <li><b>Focus on a Central Theme:</b> Stick to a main idea or theme. Too many complex elements can lead to unexpected results.</li>
    <li><b>Be Specific, But Not Overly Detailed:</b> Provide clear details but allow the AI some creative freedom.</li>
    <li><b>Experiment with Variations:</b> Slight changes in wording can produce different results. Don't hesitate to try different versions of your prompt.</li>
    <li><b>Understand the AI's Style:</b> Familiarize yourself with the types of images the AI generates well.</li>
    <li><b>Use Examples:</b> Example prompts can inspire and guide your own creations.</li>
</ul>
<p><i>Example Prompt:</i> 'A serene beach with playful dolphins, under a sunset sky, in a vibrant and cheerful style.'</p>
"""

instructions_html_widget = widgets.HTML(value=instruction_html)

# Interactive elements for detailed prompt crafting
scene_description = widgets.Textarea(description='Scene:', placeholder='e.g., "A serene beach"')
colors_emotions = widgets.Text(description='Colors/Emotions:', placeholder='e.g., "Vibrant and cheerful"')
style_artist = widgets.Text(description='Style/Artist:', placeholder='e.g., "Whimsical style"')
key_elements = widgets.Text(description='Key Elements:', placeholder='e.g., "Playful dolphins, sunset sky"')

# Button to combine inputs and generate the prompt
combine_button = widgets.Button(description='Generate Prompt', button_style='success')

# Output area for the combined prompt
combined_prompt_output = widgets.Output()

# Function to combine inputs into a prompt
def on_combine_button_clicked(b):
    combined_prompt = f"{scene_description.value}, {colors_emotions.value}, {style_artist.value}, {key_elements.value}"
    with combined_prompt_output:
        clear_output(wait=True)
        print("Generated Prompt:", combined_prompt)
        prompt_input.value = combined_prompt  # Update the main prompt input

combine_button.on_click(on_combine_button_clicked)

# Display the interactive elements for prompt crafting
prompt_crafting_ui = widgets.VBox([
    instructions_html_widget,
    scene_description,
    colors_emotions,
    style_artist,
    key_elements,
    combine_button,
    combined_prompt_output
])

display(prompt_crafting_ui)


**THE ITERATION STAGE: Try prompting again**

In [33]:
import ipywidgets as widgets
from IPython.display import clear_output, display
import datetime
import os

# Textarea for custom prompts (updated by Cell 3)
prompt_input = widgets.Textarea(
    placeholder='Your crafted prompt will appear here',
    description='Prompt:',
    layout=widgets.Layout(width='100%', height='100px'),
    disabled=True  # Disabled as it's updated by Cell 3
)

# Button to generate art
generate_button = widgets.Button(
    description='Generate Art',
    button_style='info',
    icon='check'
)

# Output area to display the generated art
output_area = widgets.Output()

# Gallery to display thumbnails of generated images
gallery_area = widgets.VBox([])

# Function to save the generated image
def save_image(img):
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"generated_image_{timestamp}.png"
    img.save(filename)
    return filename

# Function to display a full-size image when a thumbnail is clicked
def on_thumbnail_clicked(b):
    with output_area:
        clear_output(wait=True)
        img = open(b.filename, "rb").read()
        display(widgets.Image(value=img, format='png', width=500, height=500))

# Function to add a thumbnail to the gallery
def add_to_gallery(filename):
    thumbnail = widgets.Image(value=open(filename, "rb").read(), format='png', width=100, height=100)
    button = widgets.Button(description="View", button_style='primary')
    button.filename = filename
    button.on_click(on_thumbnail_clicked)
    gallery_item = widgets.HBox([thumbnail, button])
    gallery_area.children = list(gallery_area.children) + [gallery_item]

# Function to display and save generated image
def display_and_save_image(img):
    with output_area:
        display(img)
    filename = save_image(img)
    add_to_gallery(filename)

# Function to handle button click for generating art
def on_generate_button_clicked(b):
    with output_area:
        clear_output(wait=True)
        img = generate_art(prompt_input.value, width_slider.value, height_slider.value, steps_slider.value, cfg_scale_slider.value)
        if img is not None:
            display_and_save_image(img)
        else:
            print("No image was generated. Please try a different prompt.")

generate_button.on_click(on_generate_button_clicked)

# Layout for UI elements
ui_layout = widgets.Layout(
    display='flex',
    flex_flow='row wrap',
    align_items='flex-start',
    width='100%'
)

# Display the UI
ui_elements = widgets.VBox([
    theme_dropdown,
    prompt_input,
    widgets.Label("Your detailed prompt from above will be used for image generation."),
    width_slider,
    height_slider,
    steps_slider,
    cfg_scale_slider,
    generate_button,
    output_area,
    widgets.HTML('<h3>Generated Image Gallery:</h3>'),
    gallery_area
], layout=ui_layout)

display(ui_elements)